In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
from statistics import median, mean
from plotly import tools
from selfpkg import readDB, MinToMS
from dataFilter import butter_lowpass_filter, butter_highpass_filter

In [2]:
def transVolt(df):
    for i in range(8):
        df['flapper' + str(i)] = 1000 / df['flapper' + str(i)] - 10

In [3]:
csv_database = 'sqlite:///SouthHarrison.db' 
df = readDB(csv_database)

In [4]:
transVolt(df)

In [5]:
#function for generate flapper Chart
def flapperHistogramChart(df, start=None, end=None, filename='flapperHistogramChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    y = []
    for i in range(8):
        tmp = butter_highpass_filter(butter_lowpass_filter(df['flapper' + str(i)][startIndex: endIndex], 5, 50, 5), 1, 50, 5)
        for i in range(len(tmp)):
            if tmp[i] < 0.5 and tmp[i] > -0.5:
                tmp[i] = 0
        y.append(tmp)
      
    peaksPosIndex = []
    peaksNegIndex = []
    for i in range(len(y)):    
        peaksPos, _ = ss.find_peaks(y[i])#, height = h, threshold = None, distance = 20)
        peaksNeg, _ = ss.find_peaks(y[i])
        peaksPosIndex.append(peaksPos)
        peaksNegIndex.append(peaksNeg)
    
    sizeMag = 0.5
    fig = tools.make_subplots(rows=8, cols=1)
    for i in range(len(y)):
        if i == 3:
            continue
        trace0 = go.Histogram(
            x=y[i][peaksPosIndex[i]],
            name='Flapper' + str(i) + '+',
            xbins=dict(
                start=np.ndarray.min(y[i][peaksPosIndex[i]]),
                end=np.ndarray.max(y[i][peaksPosIndex[i]]),
                size=sizeMag
            ),
            marker=dict(
                color='#FFD7E9',
            ),
            opacity=1
        )
        trace1 = go.Histogram(
            x=y[i][peaksNegIndex[i]],
            name='Flapper' + str(i) + '-',
            xbins=dict(
                start=np.ndarray.min(y[i][peaksNegIndex[i]]),
                end=np.ndarray.max(y[i][peaksNegIndex[i]]),
                size=sizeMag
            ),
            marker=dict(
                color='#FFD7E9',
            ),
            opacity=1
        )
        fig.append_trace(trace0, i+1, 1)
        fig.append_trace(trace1, i+1, 1)
        fig['layout']['xaxis' + str(i+1)].update(range=[-5, 5])
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate
    
flapperHistogramChart(df, 13, 175, filename='flapperHistogramChart')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]
[ (7,1) x7,y7 ]
[ (8,1) x8,y8 ]



In [143]:
#function for generate flapper Chart
def flapperProcessedChart(df, start=None, end=None, filename='flapperProcessedChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
#     mask = [-1, 1] 
    y = []
    for i in range(8):
        tmp = butter_highpass_filter(butter_lowpass_filter(df['flapper' + str(i)][startIndex: endIndex], 5, 50, 5), 1, 50, 5)
        #tmp = df['flapper' + str(i)][startIndex: endIndex]
        for i in range(len(tmp)):
            if tmp[i] < 0.5 and tmp[i] > -0.5:
                tmp[i] = 0
        y.append(tmp)
    
#     windowSize = 50
#     for element in y:
#         for i in range(len(element)):
#             if i - 25 < 0:
#                 avg = mean(element[0: i+25])
#             else:
#                 if i + 25 > len(element) - 1:
#                     avg = mean(element[i-25: len(element) - 1])
#                 else:
#                     avg = mean(element[i-25: i+25])
                
#             element[i] = element[i] / avg
    
    timeConversionVal = 60000
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=y[0] + 60,
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[1] + 30,
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[2] + 10,
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[4] - 30,
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[5],
        name = 'flapper5',
        line = dict(
            color = ('rgb(255, 127, 0)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[6] - 90,
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[7] - 120,
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )
    
    data = [trace0, trace1, trace2, trace4, trace5, trace6, trace7]

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Friction', range = [-150, 100])
                      )

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate
    
    return y

In [144]:
list = flapperProcessedChart(df, 13, 175)

In [93]:
print(list[0][1:2])

[0.1675734]


In [132]:
#function for generate flapper Chart
def jointFeatureChart(list, start=None, end=None, filename='jointFeatureChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
        
        
    time0 = [47.47443, 51.7274, 51.80763, 55.40382, 56.77195]
    time1 = [34.17505, 34.25785, 34.42123, 34.66742, 102.1015]
    time2 = [69.65865, 69.83122, 70.26493, 71.05187, 117.1889]
    time3 = [0]
    time4 = [75.99018, 76.00637, 61.8568, 63.22452, 64.0667]
    time5 = [28.18855, 28.02753, 46.47379, 46.556965, 47.55998]
    time6 = [57.81087, 59.01242, 65.48548, 65.98377, 66.57055]
    time7 = [47.3095, 46.89328, 47.55893, 52.21448, 124.6833]
    
    time = [time0, time1, time2, time3, time4, time5, time6, time7]
    
    z = []
    for i in range(8):
        sublist = []
        for t in time[i]:
            t = MinToMS(t, df)
            index = df.index[df['time'] == t].tolist()[0]
            index = index - startIndex
            sublist.append(list[i][index-40: index+40])
        z.append(sublist)
    
    #print(z)
    res = []
    for flapper in z:
        subres = []
        for i in range(len(flapper[0])):
            tmp = 0
            for j in range(len(flapper)):
                tmp = tmp + flapper[j][i]
            subres.append(tmp/len(flapper))
        res.append(subres)

    timeConversionVal = 60000
    
    trace0 = go.Scattergl(
        x=np.arange(0, len(res[0]), 1), 
        y=[x for x in res[0]],
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=np.arange(0, len(res[1]), 1),
        y=[x for x in res[1]],
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=np.arange(0, len(res[2]), 1),
        y=[x for x in res[2]],
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    
    trace4 = go.Scattergl(
        x=np.arange(0, len(res[4]), 1),
        y=[x for x in res[4]],
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    
    trace5 = go.Scattergl(
        x=np.arange(0, len(res[5]), 1),
        y=[x for x in res[5]],
        name = 'flapper5',
        line = dict(
            color = ('rgb(255, 127, 0)'),
            width = 1)
    )
    
    trace6 = go.Scattergl(
        x=np.arange(0, len(res[6]), 1),
        y=[x for x in res[6]],
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    
    trace7 = go.Scattergl(
        x=np.arange(0, len(res[7]), 1),
        y=[x for x in res[7]],
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )
    
#     data = [trace0, trace1, trace2, trace4, trace5, trace6, trace7]

#     layout = dict(title = filename,
#                        xaxis = dict(title = 'Time(minutes)'),
#                        yaxis = dict(title = 'Friction')
#                       )

#     fig = dict(data = data, layout = layout)
    fig1 = tools.make_subplots(rows=3, cols=1)
    fig1.append_trace(trace0, 1, 1)
    fig1.append_trace(trace1, 2, 1)
    fig1.append_trace(trace2, 3, 1)
    
    plotly.offline.plot(fig1, filename = filename + '01' + '.html') #offline generate
    
    fig2 = tools.make_subplots(rows=4, cols=1)
    fig2.append_trace(trace4, 1, 1)
    fig2.append_trace(trace5, 2, 1)
    fig2.append_trace(trace6, 3, 1)
    fig2.append_trace(trace7, 4, 1)
    
    plotly.offline.plot(fig2, filename = filename + '02' + '.html') #offline generate

In [133]:
jointFeatureChart(list, 13, 175)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

